In [35]:
import pandas as pd
import datetime as dt
import numpy as np
#import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier, BaggingRegressor, AdaBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from time import time
from nltk.corpus import stopwords

In [36]:
import re
import string
import nltk
from nltk.corpus import stopwords

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [8]:
from nltk.stem.snowball import SnowballStemmer

In [9]:
stemmer=SnowballStemmer(language='english')

In [10]:
def stemmizar_texto(texto):
    texto=' '.join([stemmer.stem(palabra) for palabra in texto.split() ])
    return texto

In [111]:
nltk.download('stopwords')
english_stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
train_set=pd.read_csv("train.csv",encoding='utf-8')
test_set=pd.read_csv("test.csv",encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [98]:
def quitar_stopwords(texto):
    texto=' '.join([word for word in texto.split() if word not in english_stopwords])
    return texto

def quitar_menciones(texto):
    texto=' '.join([palabra for palabra in texto.split() if '@' not in palabra])
    return texto

In [99]:
Diccionario_de_lenguaje_de_internet={
"2moro": "Tomorrow",
"2nite": "Tonight",
"4EAE": "For Ever And Ever",
"ABT":"About",
"ADN": "Any Day Now",
"AFAIC": "As Far As I’m Concerned",
"AFAICT": "As Far As I Can Tell",
"AFAIK": "As Far As I Know",
"AFAIR": "As Far As I Remember",
"AKA": "Also Known As",
"AMA": "Ask Me Anything",
"ASAIC": "As Soon As I Can",
"ASAP": "As Soon As Possible",
"ATM": "At The Moment",
"B4": "Before",
"B4N": "Bye For Now",
"Bae": "Babe/Before Anyone Else",
"BBL": "Be Back Later",
"BBT": "Be Back Tomorrow",
"BCNU": "Be Seeing You",
"BD": "Big Deal",
"BF": "Boyfriend",
"BFF": "Best Friends Forever",
"BMT": "Before My Time",
"BOL": "Be On Later",
"BOT": "Back On Topic",
"BRB": "Be Right Back",
"BRO": "Brother",
"BT": "But",
"BTW": "By The Way",
"CFY": "Calling For You",
"CU": "See You",
"CUL": "See You Later",
"Cuz": "Because",
"CYA": "Cover Your Ass",
"DAE": "Does Anyone Else",
"DBA": "Doing Business As",
"DFTBA": "Don’t Forget To Be Awesome",
"DIKU": "Do I Know You",
"DM": "Direct Message",
"DND": "Do Not Disturb",
"DR": "Double Rainbow",
"DWBH": "Don’t Worry, Be Happy",
"ELI5": "Explain Like I’m 5",
"EOM": "End Of Message",
"EOS": "End Of Story",
"F2F": "Face To Face",
"FAQ": "Frequently Asked Question",
"FB": "Facebook",
"FBF": "Flash Back Friday",
"FF": "Follow Friday",
"FIFY": "Fixed It For You",
"FITB": "Fill In The Blank",
"FML": "Fuck My Life",
"FOMO": "Fear Of Missing Out",
"FTFY": "Fixed That For You",
"FTL": "For The Loss",
"FTW": "For The Win",
"FWB": "Friends With Benefits",
"FWIW": "For What It’s Worth",
"FYE": "For Your Entertainment",
"FYEO": "For Your Eyes Only",
"FYI": "For Your Information",
"GA": "Go Ahead",
"GAL": "Get A Life",
"GF": "Girlfriend",
"GM": "Good Morning",
"GN": "Good Night",
"Gr8": "Great",
"GTR": "Getting Ready",
"HAND": "Have A Nice Day",
"HB": "Hurry Back",
"HBD": "Happy Birthday",
"HBU": "How About You",
"HMB": "Hit Be Back",
"HMU": "Hit Me Up",
"HRU": "How Are You",
"HTH": "Hope This Helps",
"IAC": "In Any Case",
"IC": "I See",
"ICYMI": "In Case You Missed It",
"IDC": "I Don’t Care",
"IDK": "I Don’t Know",
"IG": "Instagram",
"IIRC": "If I Remember Correctly",
"IKR": "I Know Right",
"ILY": "I Love You",
"IMHO": "In My Humble Opinion",
"IMMD": "It Made My Day",
"IMY": "I Miss You",
"IRL": "In Real Life",
"IS": "I’m Sorry",
"ISO": "In Search Of",
"IU2U": "It’s Up To You",
"J4F": "Just For Fun",
"JAM": "Just A Minute",
"JFY": "Just For You",
"JIC": "Just In Case",
"JK": "Just Kidding",
"JSYK": "Just So You Know",
"KK": "Okay",
"L8": "Late",
"L8R": "Later",
"LMA": "Leave Me Alone",
"LMAO": "Laughing My Ass Off",
"LMBO": "Laughing My Butt Off",
"LMK": "Let Me Know",
"LOL": "Laugh Out Loud",
"LTNS": "Long Time No See",
"LYLAS": "Love You Like A Sister",
"M/F": "Male or Female",
"M8": "Mate",
"MP": "My pleasure",
"MSM": "Mainstream Media",
"MU": "Miss You",
"MYOB": "Mind Your Own Business",
"NAGI": "Not A Good Idea",
"NBD": "No Big Deal",
"NE1": "Anyone",
"NM": "Not Much",
"NP": "No Problem",
"NSFL": "Not Safe For Life",
"NSFW": "Not Safe For Work",
"NTS": "Note To Self",
"NVM": "Never Mind",
"OC": "Original Content",
"OH": "Overheard",
"OIC": "Oh ! I See",
"OMD": "Oh My Damn",
"OMG": "Oh My Goodness",
"OMW": "On My Way",
"OT": "Off Topic",
"OFC": "Of course",
"PAW": "Parents Are Watching",
"Pls": "Please",
"POTD": "Photo Of The Day",
"POV": "Point Of View",
"PPL": "People",
"PTB": "Please Text Back",
"Q4U": "Question For You",
"QQ": "Crying",
"RBTL": "Read Between The Lines",
"RIP": "Rest In Peace",
"RL": "Real Life",
"ROFL": "Rolling On the Floor Laughing",
"RT": "Retweet",
"RTM": "Read The Manual",
"SIS": "Sister",
"SITD": "Still In The Dark",
"SM": "Social Media",
"SMH": "Shaking My Head",
"SMY": "Somebody",
"SNH": "Sarcasm Noted Here",
"SOL": "Sooner Or Later",
"Some1": "Someone",
"SRSLY": "Seriously",
"STBY": "Sucks To Be You",
"Str8": "Straight",
"SYS": "See You Soon",
"TBA": "To Be Announced",
"TBH": "To Be Honest",
"TBT": "Throwback Thursday",
"TBT": "Truth Be Told",
"TFH": "Thread From Hell",
"TFTI": "Thanks For The Invite",
"TGIF": "Thank God It’s Friday",
"THX": "Thanks",
"TIA": "Thanks in Advance",
"TIL": "Today I Learned",
"TIME": "Tears In My Eyes",
"TL;DR": "Too Long; Didn’t Read",
"TLDR":"Too long didn’t read",
"TL DR":"Too long didn’t read",
"TLC": "Tender Loving Care",
"TMI": "Too Much Information",
"TTYL": "Talk To You Later",
"TTYS": "Talk To You Soon",
"Txt": "Text",
"TYVM": "Thank You Very Much",
"U": "You",
"U4F": "You Forever",
"UR": "Your",
"VBG": "Very Big Grin",
"VSF": "Very Sad Face",
"WB": "Welcome Back",
"WBU": "What About You?",
"WEG": "Wicked Evil Grin",
"WKND": "Weekend",
"WOM": "Word of Mouth",
"WOTD": "Word Of The Day",
"Wru": "Who Are You",
"WTH": "What The Heck?",
"WTPA": "Where The Party At?",
"WU?": "What's Up",
"WU":"What's Up",
"WYCM": "Will You Call Me?",
"WYWH": "Wish You Were Here",
"XOXO": "Hugs and Kisses",
"YGM": "You’ve Got Mail",
"YNK": "You Never Know",
"YOLO": "You Only Live Once",
"YT": "YouTube",
"YW": "You’re Welcome",
}

In [100]:
def reemplazar_lenguaje_internet(texto):
    texto=texto.upper()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=Diccionario_de_lenguaje_de_internet.get(palabra,'not internet slang')
        if(traduccion!='not internet slang'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [101]:
diccionario_contracciones = {  "ain't": "is not",
                               "aint":"is not",
                               "aren't": "are not",
                               "arent":"are not",
                               "can't": "can not",
                               "cant":"can not",
                               "cause": "because",
                               "could've": "could have",
                               "couldve":"could have",
                               "couldn't": "could not",
                               "couldnt":"could not",
                               "didn't": "did not", 
                               "didnt":"did not",
                               "doesn't": "does not",
                               "doesnt":"does not",
                               "don't": "do not",
                               "dont":"do not",
                               "hadn't": "had not",
                               "hadnt":"had not",
                               "hasn't": "has not",
                               "hasnt":"has not",
                               "haven't": "have not",
                               "havent":"have not",
                               "he'd": "he would",
                               "hed":"he would",
                               "he'll": "he will",
                               "he ll":"he will",
                               "he's": "he is",
                               "hes":"he is",
                               "how'd": "how did",
                               "howd":"how did",
                               "how'd'y": "how do you",
                               "howdy":"how do you",
                               "how'll": "how will",
                               "howll":"how will",
                               "how's": "how is",
                               "hows":"how is",
                               "I'd": "I would",
                               "id":"i would",
                               "I'd've": "I would have",
                               "idve":"i would have",
                               "I'll": "I will", 
                               "Ill":"i will", #duda: si algun tweet habla de un enfermo puede traer ruido
                               "I'll've": "I will have",
                               "I'm": "I am", 
                               "im":"i am",
                               "I've": "I have",
                               "ive":"i have",
                               "i'd": "i would",
                               "i'd've": "i would have",
                               "i'll": "i will",
                               "i'll've": "i will have",
                               "i'm": "i am", 
                               "im":"i am",
                               "i've": "i have",
                               "isn't": "is not",
                               "isnt":"is not",
                               "it'd": "it would",
                               "itd":"it would",
                               "it'd've": "it would have",
                               "it'll": "it will",
                               "itll":"it will",
                               "it'll've": "it will have",
                               "it's": "it is",
                               "its":"it is",
                               "let's": "let us",
                               "lets":"let us",
                               "ma'am": "madam",
                               "maam":"madam",
                               "mayn't": "may not",
                               "maynt":"may not",
                               "might've": "might have",
                               "mightve":"might have",
                               "mightn't": "might not",
                               "mightnt":"might not",
                               "mightn't've": "might not have",
                               "must've": "must have",
                               "mustve":"must have",
                               "mustn't": "must not",
                               "mustnt":"must not",
                               "mustn't've": "must not have",
                               "needn't": "need not",
                               "neednt":"need not",
                               "needn't've": "need not have",
                               "o'clock": "of the clock",
                               "oclock":"of the clock",
                               "oughtn't": "ought not",
                               "oughtnt":"ought not",
                               "oughtn't've": "ought not have",
                               "shan't": "shall not",
                               "shant":"shall not",
                               "sha'n't": "shall not",
                               "shan't've": "shall not have",
                               "she'd": "she would",
                               "shed":"she would",
                               "she'd've": "she would have",
                               "she'll": "she will",
                               "shell":"she will",#nuevamente aca tengo ruido por dos palabras igual escritas
                               "she'll've": "she will have", 
                               "she's": "she is",
                               "shes":"she is",
                               "should've": "should have",
                               "shouldve":"should have",
                               "shouldn't": "should not",
                               "shouldnt": "should not",
                               "shouldn't've": "should not have",
                               "so've": "so have",
                               "so's": "so as",
                               "this's": "this is",
                               "that'd": "that would",
                               "that'd've": "that would have",
                               "that's": "that is",
                               "thats":"that is",
                               "there'd": "there would",
                               "thered":"there would",
                               "there'd've": "there would have",
                               "there's": "there is",
                               "theres":"there is",
                               "here's": "here is",
                               "heres":"here is",
                               "they'd": "they would",
                               "theyd":"they would",
                               "they'd've": "they would have",
                               "they'll": "they will",
                               "theyll":"they will",
                               "they'll've": "they will have", 
                               "they're": "they are",
                               "theyre":"they are",
                               "they've": "they have", 
                               "theyve":"they have",
                               "to've": "to have", 
                               "tove":"to have",
                               "wasn't": "was not",
                               "wasnt":"was not",
                               "we'd": "we would",
                               "wed":"we would",#aca nuevamente hay conflicto
                               "we'd've": "we would have",
                               "we'll": "we will",
                               "we'll've": "we will have",
                               "we're": "we are",
                               "we've": "we have",
                               "weren't": "were not",
                               "werent":"were not",
                               "what'll": "what will",
                               "whatll":"what will",
                               "what'll've": "what will have",
                               "what're": "what are", 
                               "whatre":"what are",
                               "what's": "what is",
                               "whats":"what is",
                               "what've": "what have",
                               "whatve":"what have",
                               "when's": "when is",
                               "whens":"when is",
                               "when've": "when have",
                               "whenve":"when have",
                               "where'd": "where did",
                               "whered":"where did",
                               "where's": "where is",
                               "wheres":"where is",
                               "where've": "where have",
                               "whereve":"where have",
                               "who'll": "who will", 
                               "wholl":"who will",
                               "who'll've": "who will have",
                               "who's": "who is",
                               "whos":"who is",
                               "who've": "who have",
                               "whove":"who have",
                               "why's": "why is",
                               "whys":"why is",
                               "why've": "why have",
                               "whyve":"why have",
                               "will've": "will have",
                               "willve":"will have",
                               "won't": "will not",
                               "wont":"will not",
                               "won't've": "will not have",
                               "would've": "would have",
                               "wouldve":"would have",
                               "wouldn't": "would not",
                               "wouldnt":"would not",
                               "wouldn't've": "would not have",
                               "y'all": "you all",
                               "yall":"you all",
                               "y'all'd": "you all would",
                               "yalld":"you all would",
                               "y'all'd've": "you all would have",
                               "y'all're": "you all are",
                               "yallre":"you all are",
                               "y'all've": "you all have",
                               "you'd": "you would",
                               "youd":"you would",
                               "you'd've": "you would have",
                               "you'll": "you will",
                               "youll":"you will",
                               "you'll've": "you will have",
                               "you're": "you are",
                               "youre":"you are",
                               "you've": "you have",
                               "youve":"you have"}


In [102]:
def reemplazar_contracciones(texto):
    texto=texto.lower()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=diccionario_contracciones.get(palabra,'no es contraccion')
        if(traduccion!='no es contraccion'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [103]:
def eliminar_palabras_de_una_sola_letra(texto):
    texto=texto.lower()
    lista_palabras=texto.split()
    palabras_procesadas=[]
    for palabra in lista_palabras:
        if(len(palabra)!=1):
            palabras_procesadas.append(palabra)
    texto=" ".join([word for word in palabras_procesadas])
    return texto

In [104]:
#es necesario importar la libreria re (regular expression) para poder usar esta funcion
#ya esta bajada en el comienzo del notebook pero si lo que quieren es analizar la documentación
#lo pueden revisar en google
#falta borrar los arroba
def limpiar_texto(texto):
    texto = reemplazar_lenguaje_internet(texto)
    texto=reemplazar_contracciones(texto)
    texto=texto.lower()
    texto = re.sub('\[.*?\]', '', texto) # remove text in square brackets
    texto = re.sub('https?://\S+|www\.\S+', '', texto) # remove URLs
    texto = re.sub('<.*?>+', '', texto) # remove html tags
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # remove punctuation
    texto = re.sub('\n', '', texto) # remove words conatinaing numbers
    texto = re.sub('\w*\d\w*', '', texto)
    texto = re.sub('[‘’“”…]', '', texto)
    texto = quitar_stopwords(texto)
    texto = quitar_menciones(texto)
    texto = eliminar_palabras_de_una_sola_letra(texto)
    return texto

In [105]:
from nltk import WordNetLemmatizer

In [106]:
lemmatizer=WordNetLemmatizer()

In [107]:
def lemmatizar_texto(texto):
    lista_palabras=nltk.word_tokenize(texto)
    texto=' '.join([lemmatizer.lemmatize(word) for word in lista_palabras])
    return texto
#nota: la función esta definida aca, pero para ver los resultados que produce hay
#que ir al comienzo del notebook y ejecutar la celda de lemmatizar texto luego
#de haber limpiado el texto original

In [108]:
from nltk.corpus import wordnet

def get_wordnet_pos(palabra):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([palabra])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [109]:
def lemmatizar_con_pos(texto):
    lista_palabras=texto.split()
    texto=' '.join(lemmatizer.lemmatize(palabra,get_wordnet_pos(palabra)) for palabra in lista_palabras)
    return texto

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


### Feature: Cantidad de caracteres usados en tweets

In [184]:
cuenta_caract = train_set[['id','text']]
cuenta_caract['caracteres_usados'] = cuenta_caract['text'].str.len()
cuenta_caract_t = test_set[['id','text']]
cuenta_caract_t['caracteres_usados'] = cuenta_caract_t['text'].str.len()

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [185]:
cuenta_caract = cuenta_caract[['id','caracteres_usados']]
cuenta_caract_t = cuenta_caract_t[['id','caracteres_usados']]

In [186]:
train_set=train_set.merge(cuenta_caract,on='id',how='left')

In [187]:
test_set=test_set.merge(cuenta_caract_t,on='id',how='left')

### Feature: cantidad de menciones por tweet

In [188]:
cant_menciones = train_set[['id','text']]
cant_menciones['menciones_realizadas']=cant_menciones['text'].str.count('@')
cant_menciones_t = test_set[['id','text']]
cant_menciones_t['menciones_realizadas']=cant_menciones_t['text'].str.count('@')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [189]:
cant_menciones = cant_menciones[['id','menciones_realizadas']]
cant_menciones_t=cant_menciones_t[['id','menciones_realizadas']]

In [190]:
train_set=train_set.merge(cant_menciones,on='id',how='left')

In [191]:
test_set=test_set.merge(cant_menciones_t,on='id',how='left')

### Feature: ID que comparten localización

In [192]:
tiene_localizacion = train_set[['id','location']]
tiene_localizacion_t = test_set[['id','location']]

In [193]:
tiene_localizacion['location'].loc[~(tiene_localizacion['location'].isnull())]=1
tiene_localizacion_t['location'].loc[~(tiene_localizacion_t['location'].isnull())]=1

C:\Users\GyL\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [194]:
tiene_localizacion.fillna(0,inplace=True)
tiene_localizacion_t.fillna(0,inplace=True)

C:\Users\GyL\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [195]:
tiene_localizacion['location'].value_counts()

1    5080
0    2533
Name: location, dtype: int64

In [196]:
tiene_localizacion = tiene_localizacion.rename(columns={'location':'permite_location'})
tiene_localizacion_t = tiene_localizacion_t.rename(columns={'location':'permite_location'})

In [197]:
train_set=train_set.merge(tiene_localizacion,on='id',how='left')

In [198]:
test_set=test_set.merge(tiene_localizacion_t,on='id',how='left')

### Feature: usa Keyword

In [199]:
usa_keyword = train_set[['id','keyword']]
usa_keyword['keyword'].loc[~(usa_keyword['keyword'].isnull())] = 1
usa_keyword_t = test_set[['id','keyword']]
usa_keyword_t['keyword'].loc[~(usa_keyword_t['keyword'].isnull())] = 1

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [200]:
usa_keyword.fillna(0,inplace=True)
usa_keyword_t.fillna(0,inplace=True)

In [201]:
usa_keyword = usa_keyword.rename(columns={'keyword':'use_keyword'})
usa_keyword_t = usa_keyword_t.rename(columns={'keyword':'use_keyword'})

In [202]:
train_set = train_set.merge(usa_keyword,on='id',how='left')

In [203]:
test_set=test_set.merge(usa_keyword_t,on='id',how='left')

### Feature: Cita URL en Tweet

In [204]:
usa_url = train_set[['id','text']]
usa_url_t = test_set[['id','text']]

In [205]:
usa_url['cita_url'] = usa_url['text'].str.count('http')
usa_url_t['cita_url'] = usa_url_t['text'].str.count('http')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [206]:
usa_url = usa_url[['id','cita_url']]
usa_url_t = usa_url_t[['id','cita_url']]

In [207]:
train_set = train_set.merge(usa_url,on='id',how='left')

In [208]:
test_set=test_set.merge(usa_url_t,on='id',how='left')

### Feature: usa Hashtag

In [209]:
usa_hashtag = train_set[['id','text']]
usa_hashtag['use_hashtag']=usa_hashtag['text'].str.count('#')
usa_hashtag_t = test_set[['id','text']]
usa_hashtag_t['use_hashtag']=usa_hashtag_t['text'].str.count('#')

C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GyL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [210]:
usa_hashtag = usa_hashtag[['id','use_hashtag']]
usa_hashtag_t = usa_hashtag_t[['id','use_hashtag']]

In [683]:
train_set = train_set.merge(usa_hashtag,on='id',how='left')

In [684]:
test_set=test_set.merge(usa_hashtag_t,on='id',how='left')

### Feature: Cantidad de palabras

In [685]:
cant_palabras = train_set[['id', 'text']]
cant_palabras['cant_palabras'] = cant_palabras['text'].str.count(' ') + 1
cant_palabras_t = test_set[['id', 'text']]
cant_palabras_t['cant_palabras'] = cant_palabras_t['text'].str.count(' ') + 1

<ipython-input-685-9a582d2d4513>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_palabras['cant_palabras'] = cant_palabras['text'].str.count(' ') + 1
<ipython-input-685-9a582d2d4513>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_palabras_t['cant_palabras'] = cant_palabras_t['text'].str.count(' ') + 1


In [686]:
cant_palabras = cant_palabras[['id', 'cant_palabras']]
cant_palabras_t = cant_palabras_t[['id', 'cant_palabras']]

In [687]:
train_set = train_set.merge(cant_palabras,on='id',how='left')
test_set = test_set.merge(cant_palabras_t,on='id',how='left')

### Feature: Cantidad de abreviaciones de internet

In [688]:
def contar_abreviaciones(data):
    lista_cant = []
    for tweet in data:
        cant = 0
        tweet = quitar_stopwords(tweet)
        tweet = tweet.upper()
        tweet = tweet.split()
        for word in tweet:
            if (word in Diccionario_de_lenguaje_de_internet):
                cant = cant + 1
        lista_cant = lista_cant + [cant]
    return lista_cant

In [689]:
cant_abreviaciones = train_set[['id', 'text']]
cant_abreviaciones['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones['text'])
cant_abreviaciones_t = test_set[['id', 'text']]
cant_abreviaciones_t['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones_t['text'])

<ipython-input-689-1cd6de0a6bdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_abreviaciones['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones['text'])
<ipython-input-689-1cd6de0a6bdd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cant_abreviaciones_t['cant_abreviaciones'] = contar_abreviaciones(cant_abreviaciones_t['text'])


In [690]:
cant_abreviaciones = cant_abreviaciones[['id', 'cant_abreviaciones']]
cant_abreviaciones_t = cant_abreviaciones_t[['id', 'cant_abreviaciones']]

In [691]:
train_set = train_set.merge(cant_abreviaciones,on='id',how='left')
test_set = test_set.merge(cant_abreviaciones_t,on='id',how='left')

### Feature: Location en Estados Unidos

In [692]:
lista_estados_usa = ["ALABAMA", "AL", "ALASKA", "AK", 
                     "ARIZONA", "AZ", "ARKANSAS", "AR", 
                     "CALIFORNIA", "CA", "COLORADO", "CO", 
                     "CONNECTICUT", "CT", "DELAWARE", "DE", 
                     "FLORIDA", "FL", "GEORGIA", "GA", 
                     "HAWAII", "HI", "IDAHO", "ID", 
                     "ILLINOIS", "IL", "INDIANA", "IN", 
                     "IOWA", "IA", "KANSAS", "KS", 
                     "KENTUCKY", "KY", "LOUISIANA", "LA", 
                     "MAINE", "ME", "MARYLAND", "MD", 
                     "MASSACHUSETTS", "MA", "MICHIGAN", "MI", 
                     "MINNESOTA", "MN", "MISSISSIPPI", "MS", 
                     "MISSOURI", "MO", "MONTANA", "MT", 
                     "NEBRASKA", "NE", "NEVADA", "NV", 
                     "NEW HAMPSHIRE", "NH", "NEW JERSEY", 
                     "NJ", "NEW MEXICO", "NM", "NEW YORK", 
                     "NY", "NORTH CAROLINA", "NC", "NORTH DAKOTA", 
                     "ND", "OHIO", "OH", "OKLAHOMA", "OK", 
                     "OREGON", "OR", "PENNSYLVANIA", "PA", 
                     "RHODE ISLAND", "RI", "SOUTH CAROLINA", "SC", "CAROLINA", 
                     "SOUTH DAKOTA", "SD", "TENNESSEE", "TN", 
                     "TEXAS", "TX", "UTAH", "UT", 
                     "VERMONT", "VT", "VIRGINIA", "VA", 
                     "WASHINGTON", "WA", "WEST VIRGINIA", "WV", 
                     "WISCONSIN", "WI", "WYOMING", "WY",
                     "USA", "UNITED STATES", "SAN FRANCISCO"]

In [693]:
def ver_location_usa(data):
    lista_ubicacion_usa = []
    for location in data:
        if (location is None):
            lista_ubicacion_usa += [0]
        else:
            location_usa = False
            location = str(location)
            location = location.upper()
            if (location in lista_estados_usa):
                location_usa = True
            location = location.split()
            for word in location:
                if (word in lista_estados_usa):
                    location_usa = True
                    break
            if (location_usa):
                lista_ubicacion_usa += [1]
            else:
                lista_ubicacion_usa += [0]
    return lista_ubicacion_usa

In [694]:
location_usa = train_set[['id', 'location']]
location_usa['location_usa'] = ver_location_usa(location_usa['location'])
location_usa_t = test_set[['id', 'location']]
location_usa_t['location_usa'] = ver_location_usa(location_usa_t['location'])

<ipython-input-694-aa4731089d51>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_usa['location_usa'] = ver_location_usa(location_usa['location'])
<ipython-input-694-aa4731089d51>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_usa_t['location_usa'] = ver_location_usa(location_usa_t['location'])


In [695]:
location_usa = location_usa[['id', 'location_usa']]
location_usa_t = location_usa_t[['id', 'location_usa']]

In [696]:
train_set = train_set.merge(location_usa,on='id',how='left')
test_set = test_set.merge(location_usa_t,on='id',how='left')

### Feature: Tweet tiene caritas

In [697]:
lista_caritas = [":)", ";)", ":(", ";(", "XD", "xD", "xd", ":P", ":p", "-_-", ":O", ":'(", ":D", ":-D"]

In [698]:
def buscar_caritas(data):
    tiene_carita = []
    for tweet in data:
        tweet = quitar_menciones(tweet)
        tweet = quitar_links(tweet)
        hay_carita = False
        for carita in lista_caritas:
            if (carita in tweet):
                hay_carita = True
                break
        if (hay_carita):
            tiene_carita += [1]
        else:
            tiene_carita += [0]
    return tiene_carita

In [699]:
has_emoji = train_set[['id', 'text']]
has_emoji['has_emoji'] = buscar_caritas(has_emoji['text'])
has_emoji_t = test_set[['id', 'text']]
has_emoji_t['has_emoji'] = buscar_caritas(has_emoji_t['text'])

<ipython-input-699-31913808ce48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_emoji['has_emoji'] = buscar_caritas(has_emoji['text'])
<ipython-input-699-31913808ce48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_emoji_t['has_emoji'] = buscar_caritas(has_emoji_t['text'])


In [700]:
has_emoji = has_emoji[['id', 'has_emoji']]
has_emoji_t = has_emoji_t[['id', 'has_emoji']]

In [701]:
train_set = train_set.merge(has_emoji,on='id',how='left')
test_set = test_set.merge(has_emoji_t,on='id',how='left')

### Feature: Palabras con misma letra repetida

In [702]:
lista_letras_seguidas = ["aaa", "bbb", "ccc", "ddd", "eee", "fff", "ggg", "hhh", "iii", "jjj", "kkk", "lll", "mmm", "nnn", "ooo", "ppp", "qqq", "rrr", "sss", "ttt", "uuu", "vvv", "www", "xxx", "yyy", "zzz"]

In [703]:
def buscar_letras_seguidas(data):
    tiene_letras_seguidas = []
    for tweet in data:
        tweet = quitar_menciones(tweet)
        tweet = quitar_links(tweet)
        hay_letra_seguida = False
        for letra in lista_letras_seguidas:
            if (letra in tweet):
                hay_letra_seguida = True
                break
        if (hay_letra_seguida):
            tiene_letras_seguidas += [1]
        else:
            tiene_letras_seguidas += [0]
    return tiene_letras_seguidas

In [704]:
has_repeated_letter = train_set[['id', 'text']]
has_repeated_letter['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter['text'])
has_repeated_letter_t = test_set[['id', 'text']]
has_repeated_letter_t['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter_t['text'])

<ipython-input-704-8c809b22b19a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_repeated_letter['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter['text'])
<ipython-input-704-8c809b22b19a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_repeated_letter_t['has_repeated_letter'] = buscar_letras_seguidas(has_repeated_letter_t['text'])


In [705]:
has_repeated_letter = has_repeated_letter[['id', 'has_repeated_letter']]
has_repeated_letter_t = has_repeated_letter_t[['id', 'has_repeated_letter']]

In [706]:
train_set = train_set.merge(has_repeated_letter,on='id',how='left')
test_set = test_set.merge(has_repeated_letter_t,on='id',how='left')

In [707]:
train_set.to_csv('train_data.csv')

In [708]:
test_set.to_csv('test_data.csv')

In [709]:
train_set

,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0,0,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0,0,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,83,0,0,0,1,0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,125,2,0,0,0,0
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,0,0,0,1,0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,137,0,0,0,0,0


In [214]:
test_set

,id,keyword,location,text,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag
0,0,NaN,NaN,Just happened a terrible car crash,34,0,0,0,0,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,0,0,0,0,1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,0,0,0,0,0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,0,0,0,0,2
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,55,0,0,0,0,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,139,0,0,0,0,0
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,55,0,0,0,1,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,65,0,0,0,1,0


## Feature: cantidad de palabras únicas usadas

In [14]:
train_set=pd.read_csv("train_data.csv")

In [2]:
lista_cantidad_de_palabras_unicas=[]
def contar_cantidad_de_palabras_unicas(texto):
    diccionario_auxiliar={}
    palabras=texto.split()
    for palabra in palabras:
        esta=diccionario_auxiliar.get(palabra,"no esta")
        if(esta=="no esta"):
            diccionario_auxiliar[palabra]="esta"
    lista_cantidad_de_palabras_unicas.append(len(diccionario_auxiliar))

In [ ]:
train_set['numero_de_palabras_unicas']=train_set['text'].apply

## Feature: Hace mencion a alguna ciudad o país

In [16]:
ciudades_del_mundo=pd.read_csv('world-cities_csv.csv')

In [24]:
ciudades_del_mundo.head()

,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,Ras al-Khaimah,United Arab Emirates,Raʼs al Khaymah,291074
4,Khawr Fakkān,United Arab Emirates,Ash Shāriqah,291696


In [75]:
LISTA_CIUDADES=[]
def generar_lista_ciudades(texto):
    texto=str(texto)
    texto=texto.lower()
    lista_auxiliar=texto.split()
    if len(lista_auxiliar)!=1:
        texto=' '.join([word for word in lista_auxiliar])
    global LISTA_CIUDADES
    LISTA_CIUDADES.append(texto)

In [77]:
LISTA_CIUDADES

[]

In [30]:
ciudades=ciudades_del_mundo['name']

In [32]:
ciudades=list(ciudades)

In [33]:
len(ciudades)

23018

In [73]:
def menciona_ciudad(texto):
    texto=texto.lower()
    global ciudades
    for ciudad in ciudades:
        lista_auxiliar=ciudad.split()
        if len(lista_auxiliar)!=1:
            ciudad=' '.join([word for word in lista_auxiliar])
        if ciudad.lower() in texto:
            return ciudad
    return None

In [68]:
def ciudades_por_longitud(lista_ciudades):
    diccionario={}
    for ciudad in lista_ciudades:
        longitud=len(ciudad)
        contador=diccionario.get(longitud,-1)
        if contador==-1:
            diccionario[longitud]=1
        else:
            diccionario[longitud]=contador+1
    for key in diccionario.keys():
        print("llave: " + str(key) + "   "+ str(diccionario[key]))

In [70]:
#ciudades_por_longitud(ciudades)

In [39]:
train_set=pd.read_csv('train_data.csv')

In [74]:
train_set['menciona_ciudad']=train_set['text'].apply(menciona_ciudad)

KeyboardInterrupt: 

In [44]:
train_set.loc[train_set['menciona_ciudad']!=None]

,Unnamed: 0,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag,menciona_ciudadd,menciona_ciudad
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0,0,1,Aso,Aso
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0,0,0,Ron,Ron
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0,0,0,Ati,Ati
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0,0,1,Wil,Wil
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0,0,2,Wil,Wil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,83,0,0,0,1,0,Ho,Ho
7609,7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,125,2,0,0,0,0,Wil,Wil
7610,7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,0,0,0,1,0,Wa,Wa
7611,7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,137,0,0,0,0,0,Teni,Teni


In [48]:
dicc_paises={}
def generar_lista_paises(texto):
    global dicc_paises
    texto=texto.lower()
    esta=dicc_paises.get(texto,-1)
    if esta==-1:
        dicc_paises[texto]=1


In [152]:
lista_paises=dicc_paises.keys()
def menciona_pais(texto):
    texto=texto.lower()
    for pais in lista_paises:
        if pais in texto:
            return 1
    return 0

In [57]:
ciudades_del_mundo['country'].apply(generar_lista_paises)

0        None
1        None
2        None
3        None
4        None
         ... 
23013    None
23014    None
23015    None
23016    None
23017    None
Name: country, Length: 23018, dtype: object

In [153]:
train_set['menciona_pais']=train_set['text'].apply(menciona_pais)

In [154]:
train_set.head()

,Unnamed: 0,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag,menciona_pais,numero_de_palabras,palabras_unicas,numero_palabras_desconocidas
0,0,1,NaN,NaN,deed reason earthquake may allah forgive,1,69,0,0,0,0,1,0,6,6,0
1,1,4,NaN,NaN,forest fire near la ronge sask canada,1,38,0,0,0,0,0,1,7,7,0
2,2,5,NaN,NaN,resident ask shelter place notify officer evac...,1,133,0,0,0,0,0,0,11,9,0
3,3,6,NaN,NaN,people receive wildfire evacuation order calif...,1,65,0,0,0,0,1,0,6,6,0
4,4,7,NaN,NaN,get sent photo ruby alaska smoke wildfire pour...,1,88,0,0,0,0,2,0,9,9,0


# Las siguientes features usarlas sólo despues de limpiar el texto

In [113]:
train_set['text']=train_set['text'].apply(limpiar_texto)

In [114]:
train_set['text']=train_set['text'].apply(lemmatizar_con_pos)

In [139]:
train_set['text']=train_set['text'].apply(limpiar_texto)

### Feature: cantidad de palabras en el tweet

In [95]:
def contar_cantidad_de_palabras(texto):
    contador=0
    texto=texto.lower()
    lista_palabras=texto.split()
    for palabra in lista_palabras:
        contador+=1
    return contador

In [140]:
train_set['numero_de_palabras']=train_set['text'].apply(contar_cantidad_de_palabras)

### Feature: cantidad de palabras únicas utilizadas


In [85]:
# nota: aplicar la función solamente una vez que el texto fue limpiado
def contar_cantidad_de_palabras_unicas(texto):
    diccionario_auxiliar={}
    palabras=texto.split()
    for palabra in palabras:
        esta=diccionario_auxiliar.get(palabra,"no esta")
        if(esta=="no esta"):
            diccionario_auxiliar[palabra]="esta"
    return(len(diccionario_auxiliar))

In [141]:
train_set['palabras_unicas']=train_set['text'].apply(contar_cantidad_de_palabras_unicas)

### Feature: cantidad de palabras no reconocidas

In [86]:
path_glove='/home/tomas/organizacion_de_datos/tp2/glove.6B.100d.txt'

In [87]:
diccionario_de_palabras={}
def generar_diccionario_de_palabras():
    embeddings=open(path_glove,'r',encoding='UTF-8')
    while(True):
        f=embeddings.readline()
        if(len(f)==0):
            break
        f=f.split()
        lista=f[0].split()
        diccionario_de_palabras[lista[0]]='y'
    embeddings.close()

In [88]:
generar_diccionario_de_palabras()

In [89]:
def contar_palabras_desconocidas(texto):
    global diccionario_de_palabras
    contador=0
    texto=texto.lower()
    lista_palabras=texto.split()
    for palabra in lista_palabras:
        esta=diccionario_de_palabras.get(palabra,'no esta')
        if esta=='no esta':
            contador+=1
    return contador

In [142]:
train_set['numero_palabras_desconocidas']=train_set['text'].apply(contar_palabras_desconocidas)

In [143]:
train_set.head()

,Unnamed: 0,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag,menciona_pais,numero_de_palabras,palabras_unicas,numero_palabras_desconocidas
0,0,1,NaN,NaN,deed reason earthquake may allah forgive,1,69,0,0,0,0,1,No,6,6,0
1,1,4,NaN,NaN,forest fire near la ronge sask canada,1,38,0,0,0,0,0,Si,7,7,0
2,2,5,NaN,NaN,resident ask shelter place notify officer evac...,1,133,0,0,0,0,0,No,11,9,0
3,3,6,NaN,NaN,people receive wildfire evacuation order calif...,1,65,0,0,0,0,1,No,6,6,0
4,4,7,NaN,NaN,get sent photo ruby alaska smoke wildfire pour...,1,88,0,0,0,0,2,No,9,9,0


In [131]:
test_set=pd.read_csv('test_data.csv')

In [133]:
test_set['menciona_pais']=test_set['text'].apply(menciona_pais)

In [135]:
test_set['text']=test_set['text'].apply(limpiar_texto)

In [136]:
test_set['text']=test_set['text'].apply(lemmatizar_con_pos)

In [144]:
test_set['text']=test_set['text'].apply(limpiar_texto)

In [147]:
test_set['numero_de_palabras']=test_set['text'].apply(contar_cantidad_de_palabras)

In [149]:
test_set['palabras_unicas']=test_set['text'].apply(contar_cantidad_de_palabras_unicas)

In [150]:
test_set['numero_palabras_desconocidas']=test_set['text'].apply(contar_palabras_desconocidas)

In [156]:
def convertir(texto):
    if texto=="Si":
        return 1
    return 0

In [157]:
test_set['menciona_pais']=test_set['menciona_pais'].apply(convertir)

In [158]:
test_set

,Unnamed: 0,id,keyword,location,text,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag,menciona_pais,numero_de_palabras,palabras_unicas,numero_palabras_desconocidas
0,0,0,NaN,NaN,happen terrible car crash,34,0,0,0,0,0,0,4,4,0
1,1,2,NaN,NaN,heard earthquake sorry different city stay saf...,64,0,0,0,0,1,0,8,8,0
2,2,3,NaN,NaN,sorry forest fire spot pond geese flee across ...,96,0,0,0,0,0,0,11,11,0
3,3,9,NaN,NaN,apocalypse light spokane wildfire,40,0,0,0,0,2,0,4,4,0
4,4,11,NaN,NaN,typhoon soudelor kill china taiwan,45,0,0,0,0,0,1,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,3258,10861,NaN,NaN,earthquake safety los angeles ûò safety faste...,55,0,0,0,0,0,0,8,7,2
3259,3259,10865,NaN,NaN,storm ri bad last hurricane hardest hit yard l...,139,0,0,0,0,0,0,15,15,0
3260,3260,10868,NaN,NaN,green line derailment chicago,55,0,0,0,1,0,0,4,4,0
3261,3261,10874,NaN,NaN,meg issue hazardous weather outlook hwo,65,0,0,0,1,0,0,6,6,1
